# Timestep Optimization

As we observed in the falling ball example, the size of the timestep chosen can have serious consequences on the result of a simulation. If the chosen timestep is too large, interactions can be missed (such as contact with the ground in the falling ball example). However, the smaller the chosen timestep, the longer the simulation will take. As a result, optimizing the timestep is critical to achieving optimal performance for your simulations.

Fortunately, in molecular dynamics optimization of the timestep by checking for energy conservation in the microcanonical ensemble (i.e. constant energy, volume, and number of particles). As mentioned, if the timestep is too large interactions will be missed, and one will observe a drift in energy, rather than this value being held constant.

--------

NOTE: This tutorial is an adaptation of Andrew Summers' LJ Tutorial located at
https://github.com/summeraz/chbe4830/blob/master/Assignment3/LJ-NVE.ipynb

NOTE: This tutorial has been adapted from the LJ tutorial located at https://bitbucket.org/glotzer/hoomd-examples.

NOTE: A detailed description of all HOOMD commands can be found at https://hoomd-blue.readthedocs.io/en/stable/.

## Initialize

Import the hoomd python package and the md component to execute MD simulations. Also import the `deprecated` component for outputting `hoomdxml` trajectory information. 

In [ ]:
import hoomd
import hoomd.md
import hoomd.deprecated.dump

Initialize the execution context to control where HOOMD will execute the simulation. When no command line options are provided, HOOMD will auto-select a GPU if it exists, or run on the CPU.

In [ ]:
hoomd.context.initialize("")

Initialize an $n$ by $n$ by $n$ simple cubic lattice of particles, where `a` represents the lattice constant. The lattice initializer by default creates all particles named type "A", and with 0 velocity.

In [ ]:
lattice = hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5)

Initialize particle velocities from a Gaussian distribution.

In [ ]:
import random
random.seed(1)
T_init = 0.1
for p in lattice.particles:
    p.velocity = (random.gauss(0, T_init), random.gauss(0, T_init), random.gauss(0, T_init))

## Define potential energy

$ V(r) = 4 \varepsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^{6} \right] $, where $r$ < $r$<sub>cut</sub>

In the Lennard-Jones system, pairs of particles closer than $r_\mathrm{cut}$ interact with this potential energy.

Choose the neighbor list acceleration structure to find neighboring particles efficiently. In systems with only one cutoff length, the cell method performs best.

In [ ]:
nl = hoomd.md.nlist.cell(r_buff=0.6, check_period=1)

Define the functional form of the pair interaction and evaluate using the given neighbor list acceleration structure.

In [ ]:
lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)

Specify pair potential parameters for every pair of types in the simulation.

In [ ]:
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)

## Select integrator

The integrator defines the equations of motion that govern the system of particles, given the current configuration of the particles and the net force from all potentials. The standard integration mode in HOOMD allows different integrators to apply to different groups of particles with the same step size $dt$.

In [ ]:
hoomd.md.integrate.mode_standard(dt=0.002)

Apply NVE integration using the Velocity-Verlet algorithm.

In [ ]:
all = hoomd.group.all()
hoomd.md.integrate.nve(group=all)

## Write output

The `hoomd.analyze.log` method can be used to log a variety of system properties (see http://hoomd-blue.readthedocs.io/en/stable/module-hoomd-analyze.html#hoomd.analyze.log). Here we will periodically log both the potential energy and kinetic energy of the system to a text file.

In [ ]:
hoomd.analyze.log(filename="analyze.log",
                  quantities=['potential_energy',
                              'kinetic_energy'],
                  period=100,
                  overwrite=True)

Periodically write the particle configurations to a gsd file.

In [ ]:
hoomd.dump.gsd("trajectory.gsd", period=2e4, group=all, overwrite=True)

From the `deprecated` module, output the `hoomdxml` trajectory data for easier interfacing with `MDTraj` later.

In [ ]:
hoomd.deprecated.dump.xml(all, filename='traj.xml', period=2e4, vis=True)

## Run the simulation

Take 1,000,000 steps forward in time.

In [ ]:
hoomd.run(1e6)

## Examine the output

Using matplotlib, we'll plot the total energy of our system (kinetic + potential energy) over time. If our timestep is not too large, energy will be conserved and, while we will observe some fluctuations, the value should remain essentially constant over time. If the timestep is too large, we will observe an energy drift.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('default')

data = np.genfromtxt(fname='analyze.log', skip_header=True);

In [ ]:
plt.figure(figsize=(4,2.2), dpi=140);
plt.plot(data[:,0], data[:,1] + data[:,2]);
plt.xlabel('# of timesteps');
plt.ylabel('total energy');

From the above plot, a timestep of 0.002 sufficiently prevents energy drift in our system; however, the simulation could be sped up if we increased the timestep.

#### Exercise

Increase the timestep and re-run the previous cells (you may want to just restart your kernel) until you have determined the maximum timestep that still prevents energy drift. Note, you may want to make use of a "for" loop.

## Visualization

Let's visualize the trajectory of the simulation we just performed. While tools do exist to view trajectories from within a notebook, software such as VMD and Ovito provide more robust tools for this purpose. Here, we'll pull up our simulation in VMD and observe what's going on.

In [ ]:
%%bash
vmd trajectory.gsd

## Analysis

Now lets analyze the trajectory a bit more. Using the `MDTraj` python package, we can analyze the system as it evolves over time and calculate some interesting properties. In this example, we will calculate the radial distribution function (RDF).

In [ ]:
import mdtraj as md

Lets load in two of our hoomdxml trajectories. One near the beginning when it was more solid, and one near the end of the simulation, when it was a liquid.

In [ ]:
more_solid = md.load_hoomdxml(filename="./traj.xml.0000000000.xml")
less_solid = md.load_hoomdxml(filename="./traj.xml.0000980000.xml")

# scale xyz positions
more_solid.xyz = more_solid.xyz / 10
less_solid.xyz = less_solid.xyz / 10

In [ ]:
print(more_solid)
print(less_solid)

Lets calculate the RDF's

In [ ]:
r_solid, g_r_solid = md.compute_rdf(more_solid, more_solid.top.select_pairs("all", "all"))

In [ ]:
print(g_r_solid)

In [ ]:
pyplot.plot(r_solid, g_r_solid)

In [ ]:
r_liq, g_r_liq = md.compute_rdf(less_solid, less_solid.top.select_pairs("all", "all"))

In [ ]:
print(g_r_liq)

In [ ]:
pyplot.plot(r_liq, g_r_liq)